In [2]:
import arcpy
arcpy.env.overwriteOutput = True
import arcpy.analysis
import os

In [31]:
def Clip_Raster(Folder,Raster,OutputPath):
    """
    遍历一个文件夹下的所有栅格，裁剪另外一个栅格
    """
    arcpy.env.workspace = Folder
    l = arcpy.ListRasters()
    for i in range(len(l)):
        OutputRaster = OutputPath + "\\" + l[i]
        description = arcpy.Describe(l[i])
        extent = str(description.extent.XMin) + " " + str(description.extent.YMin) + " " + str(description.extent.XMax) + " " + str(description.extent.YMax)
        arcpy.management.Clip(Raster, extent, OutputRaster, "0.tif", "#", "NONE", "MAINTAIN_EXTENT")

In [ ]:
def Clip_Rasters(Raster,InFolder,OutFolder):
    """
        似乎和上面一样
    """
    arcpy.env.workspace = InFolder
    l = arcpy.ListRasters()
    for i in range(len(l)):
        OutRaster = OutFolder + "\\" + l[i]
        description = arcpy.Describe(l[i])
        extent = str(description.extent.XMin) + " " + str(description.extent.YMin) + " " + str(description.extent.XMax) + " " + str(description.extent.YMax)
        arcpy.management.Clip(Raster, extent, OutRaster, "0.tif", "3", "NONE", "MAINTAIN_EXTENT")

In [2]:
def clip(input_data,clip_feature,worksapce):
    if input_data[-4:] == ".tif":
        with arcpy.da.SearchCursor(clip_feature,["id","SHAPE@"]) as cursor:
                for row in cursor:
                    outExtractByMask = ExtractByMask(input_data, row[1])
                    outExtractByMask.save(worksapce+"\\"+str(row[0])+".tif")
    elif input_data[-4:] == ".shp":
        with arcpy.da.SearchCursor(clip_feature,["id","SHAPE@"]) as cursor:
                for row in cursor:
                    arcpy.Clip_analysis(input_data, row[1], worksapce+"\\"+str(row[0])+".shp")



In [10]:
import arcpy.management
沟间地_C = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\沟间地\Clip"
#山体单元_C = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\山体单元\Clip"
def Delete_Empty(Workspace):
    arcpy.env.workspace = Workspace
    l = arcpy.ListFeatureClasses()
    for i in range(len(l)):
        if arcpy.management.GetCount(l[i])[0] == "0":
            arcpy.management.Delete(l[i])

In [30]:
DissolvePath = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\沟间地\Dissolve"
for i in range(len(l)):
    #arcpy.management.CalculateField(l[i], "Id", "0")
    DissolveFeature = DissolvePath + "\\" + l[i]
    arcpy.management.Dissolve(l[i],DissolveFeature,"Id")

In [33]:
import arcpy.analysis
IntersectionPath = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\沟间地\Intersection"
LandformUnitPath = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\山体单元\Clip"
for i in range(len(l)):
    F_沟间地 = DissolvePath + "\\" + l[i]
    F_山体单元 = LandformUnitPath+ "\\" + l[i]
    IntersectionFeature = IntersectionPath + "\\" + l[i]
    arcpy.analysis.Intersect([F_沟间地,F_山体单元],IntersectionFeature)
    arcpy.management.CalculateGeometryAttributes(IntersectionFeature, "area AREA", '', "SQUARE_METERS", None, "SAME_AS_INPUT")

In [44]:
import os
EliminatePath = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\沟间地\Eliminate"
arcpy.env.overwriteOutput = True
for i in range(len(l)):
    IntersectionFeature = IntersectionPath + "\\" + l[i]
    Eliminateresult = EliminatePath + "\\" + l[i]
    if os.path.exists(Eliminateresult) == False:
    
        arcpy.management.EliminatePolygonPart(IntersectionFeature, Eliminateresult, "AREA", "2100 SquareMeters", 0, "CONTAINED_ONLY")

In [45]:
SmoothPath = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\沟间地\Smooth"
for i in range(len(l)):
    Eliminateresult = EliminatePath + "\\" + l[i]
    Smoothresult = SmoothPath + "\\" + l[i]
    if os.path.exists(Smoothresult) == False:
        arcpy.cartography.SmoothPolygon(Eliminateresult, Smoothresult, "PAEK", "120 Meters", "FIXED_ENDPOINT", "FLAG_ERRORS", None)

In [3]:
Final_IntersectionPath = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\沟间地\实验区域相交"
SmoothPath = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\沟间地\Smooth"
arcpy.env.workspace = SmoothPath
l = arcpy.ListFeatureClasses()
Feature = r"D:\鲁方圆\试验区域\试验区域\试验区域.shp"
for i in range(len(l)):
    SmoothFeature = SmoothPath + "\\" + l[i]
    OutFeature = Final_IntersectionPath + "\\" +l[i]
    arcpy.analysis.Intersect([SmoothFeature ,Feature],OutFeature)
    if arcpy.management.GetCount(OutFeature)[0] == "0":
        arcpy.management.Delete(OutFeature)

In [7]:
Raster = r"D:\proproject\地貌分类模型训练\不同策略\DEM+影像+坡度+山体阴影\结果处理\result_binary.tif"
ResultPath = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\沟间地\实验区域相交"
OutputPath = r"D:\proproject\黄土地貌分类729\沟间地提取\大模型\沟间地\Result"
arcpy.env.workspace = ResultPath
l = arcpy.ListFeatureClasses()
for i in range(len(l)):
    OutRaster = OutputPath + "\\" + l[i].split(".shp")[0] + ".tif"
    out_raster = arcpy.sa.ZonalStatistics(l[i], "FID", Raster, "MAJORITY", "DATA", "CURRENT_SLICE", 90, "AUTO_DETECT", "ARITHMETIC", 360)
    out_raster.save(OutRaster )

In [ ]:
def Process(Workspace,DissolvePath,IntersectionPath,EliminatePath,SmoothPath,Final_IntersectionPath,Feature):
    """
        Workspace是处理完空要素后的地貌单元（沟间地）
        每个沟间地要素类融合成一个要素
        沟间地再与反地形汇点地貌单元的相交
        消除2100平方米
        平滑
        与实验区域Feature相交
    """
    arcpy.env.workspace = Workspace
    l = arcpy.ListFeatureClasses()
    for i in range(len(l)):
        arcpy.management.CalculateField(l[i], "Id", "0")
        DissolveFeature = DissolvePath + "\\" + l[i]
        arcpy.management.Dissolve(l[i],DissolveFeature,"Id")
        
        F_沟间地 = DissolvePath + "\\" + l[i]
        F_山体单元 = LandformUnitPath+ "\\" + l[i]
        IntersectionFeature = IntersectionPath + "\\" + l[i]
        arcpy.analysis.Intersect([F_沟间地,F_山体单元],IntersectionFeature)
        arcpy.management.CalculateGeometryAttributes(IntersectionFeature, "area AREA", '', "SQUARE_METERS", None, "SAME_AS_INPUT")

        Eliminateresult = EliminatePath + "\\" + l[i]
        if os.path.exists(Eliminateresult) == False:
            arcpy.management.EliminatePolygonPart(IntersectionFeature, Eliminateresult, "AREA", "2100 SquareMeters", 0, "CONTAINED_ONLY")
        
        Smoothresult = SmoothPath + "\\" + l[i]
        if os.path.exists(Smoothresult) == False:
            arcpy.cartography.SmoothPolygon(Eliminateresult, Smoothresult, "PAEK", "120 Meters", "FIXED_ENDPOINT", "FLAG_ERRORS", None)

        OutFeature = Final_IntersectionPath + "\\" +l[i]
        arcpy.analysis.Intersect([SmoothFeature ,Feature],OutFeature)
        if arcpy.management.GetCount(OutFeature)[0] == "0":
            arcpy.management.Delete(OutFeature)